In [9]:
# Import SQL alchemy dependencies
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func, inspect

# Import other dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')

### Reflect Tables into SQLAlchemy ORM

In [10]:
# Create engine to the hawaii database
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [11]:
# Create automap base
Base = automap_base()

# Prepare base to reflect tables
Base.prepare(engine, reflect=True)

In [12]:
# Print classes from base
Base.classes.keys()

['measurement', 'station']

In [13]:
# Save a reference to the classes
Station = Base.classes.station
Measurement = Base.classes.measurement

In [14]:
# Begin a session
session = Session(engine)

### Exploratory Precipitation Analysis

In [16]:
# Use inspector to see the class/table names (same as keys)
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [17]:
# Use inspector to see the column names for measurement
columns_measurement = inspector.get_columns('measurement')

for c in columns_measurement:
    print(c["name"], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [18]:
# Use inspector to see the column names for station
columns_station = inspector.get_columns('station')

for c in columns_station:
    print(c["name"], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [19]:
# See the first five rows of data in measurment
engine.execute("SELECT * FROM measurement LIMIT 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [22]:
# Find the most recent date in measurement
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23',)

In [32]:
# Retrieve the last 12 months of precipitation data
twelve_months = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').order_by(Measurement.date).all()



In [41]:
# Load the query results into a DataFrame
query_df = pd.DataFrame(twelve_months, columns = ["Date", "Precipitation"])
query_df.set_index("Date", inplace=True)
#query_df.sort_values("Date")


In [47]:
# Plot the results using the DataFrame plot method
query_df.plot.bar(x="Date", y="Precipitation")
#plt.tight_layout()
#plt.xticks
plt.xlabel("Date")
plt.ylabel("Inches")
plt.titel("Precipitation in the Last 12 Months")
plt.show()

KeyError: 'Date'

### Exploratory Station Analysis

In [ ]:
# Close the session
session.close()